In [9]:
import pandas as pd
import numpy as np
import tqdm

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import backend as K

# for openvino
import tensorflow as tf

In [10]:
# load mnist dataset
from keras.datasets import mnist
(x_train_img, y_train_label), (x_test_img, y_test_label) = mnist.load_data()

# preview data shape
print('train data: {} images'.format(x_train_img.shape), type(x_train_img))
print('test data: {} images'.format(x_test_img.shape), type(x_test_img))

# reshape (add color channel)
x_train_img = x_train_img.reshape(-1, 28, 28, 1)
x_test_img = x_test_img.reshape(-1, 28, 28, 1)

# check shape
print('train data after: {} images'.format(x_train_img.shape), type(x_train_img))
print('test data after: {} images'.format(x_test_img.shape), type(x_test_img))

train data: (60000, 28, 28) images <class 'numpy.ndarray'>
test data: (10000, 28, 28) images <class 'numpy.ndarray'>
train data after: (60000, 28, 28, 1) images <class 'numpy.ndarray'>
test data after: (10000, 28, 28, 1) images <class 'numpy.ndarray'>


In [11]:
# normalize x matrices
x_train = x_train_img.astype('float32') / 255
x_test = x_test_img.astype('float32') / 255

# one hot encode labels
y_train = np_utils.to_categorical(y_train_label, num_classes=10)
y_test = np_utils.to_categorical(y_test_label, num_classes=10)

print('train labels: {} images'.format(y_train.shape), type(y_train))
print('test labels: {} images'.format(y_test.shape), type(y_test))

train labels: (60000, 10) images <class 'numpy.ndarray'>
test labels: (10000, 10) images <class 'numpy.ndarray'>


In [12]:
# build keras model
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(5,5), padding='same', input_shape=(28,28,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 32)        12832     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)              

In [5]:
# train model, sae best weigths with checkpointer
checkpointer = ModelCheckpoint(filepath='best_model.h5', verbose=1, save_best_only=True)

train_history = model.fit(x=x_train, y=y_train,
                          validation_split=0.1,
                         epochs=1,
                         batch_size=200,
                         verbose=1,
                         callbacks=[checkpointer])

model.evaluate(x_test, y_test, verbose=1)


Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 99s 2ms/step - loss: 0.4049 - accuracy: 0.8743 - val_loss: 0.0701 - val_accuracy: 0.9793

Epoch 00001: val_loss improved from inf to 0.07007, saving model to best_model.h5
10000/10000 [==============================] - 8s 803us/step


[0.07475339432712644, 0.9753999710083008]

In [13]:
# keras to tf conversion
keras_model_path = 'best_model.h5'

# freeze state of a session into a pruned compilation graph
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ''
        frozen_graph = convert_variables_to_constants(session, input_graph_def, 
                                                      output_names, freeze_var_names)
        return frozen_graph

# loading keras model
K.set_learning_phase(0)
model = load_model(keras_model_path)

# create frozen graph of the keras model
frozen_graph = freeze_session(K.get_session(), 
                              output_names=[out.op.name for out in model.outputs])

# save model as .pb file
tf.train.write_graph(frozen_graph, 'tf_model', 'tf_model.pb', as_text=False)


INFO:tensorflow:Froze 132 variables.
[ INFO ] Froze 132 variables.
INFO:tensorflow:Converted 132 variables to const ops.
[ INFO ] Converted 132 variables to const ops.


'tf_model/tf_model.pb'

In [ ]:
# convert tensorflow graph to ir_model (from terminal or bash)
cd /
cd opt/intel/openvino_2019.3.376/deployment_tools/model_optimizer/
conda activate my_env
mo.py --data_type FP16 --framework tf --input_model /home/evgenii/Documents/01_These/keras_ncs2_converter/tf_model/tf_model.pb --model_name ir_model --output_dir /home/evgenii/Documents/01_These/keras_ncs2_converter/ir_model --input_shape [1,28,28,1]


In [15]:
# run inference on NCS2
import cv2
import sys
import os
import logging as log
from time import time
from openvino.inference_engine import IENetwork, IEPlugin

im_path = 'inference_image/6.jpg'

log.basicConfig(format='[ %(levelname)s ] %(message)s', level=log.INFO, stream=sys.stdout)
model_xml_path = 'ir_model/ir_model.xml'
model_bin_path = 'ir_model/ir_model.bin'

# plugin initialization for specified device
plugin = IEPlugin(device='MYRIAD')

# read ir model
log.info('loading network files: \n\t{}\n\t{}'.format(model_xml_path, model_bin_path))
net = IENetwork(model=model_xml_path, weights=model_bin_path)

log.info('Preparing input blobs')
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))

# prepare image
# get shapes
n, c, h, w = net.inputs[input_blob].shape
print(n, c, h, w)
prepimg = np.ndarray(shape=(n, c, h, w))

# read image as grayscale
im = cv2.imread(im_path, 0)

# resize image
resized_image = cv2.resize(im, (28, 28), interpolation=cv2.INTER_CUBIC)

# change data layout from HW to NCHW
prepimg[0,0,:,:] = resized_image

# loading model to plugin
log.info('Loading model to the plugin')
exec_net = plugin.load(network=net)
del net

# start sync inference
log.info('Starting inference ({} iterations)'.format(1))
infer_time = []
t0 = time()
res = exec_net.infer(inputs={input_blob: prepimg})
infer_time.append((time() - t0) * 1000)
log.info('Average running time of one iteration: {} ms'.format(np.average(np.asarray(infer_time))))

# processing output blob
log.info('Processing output blob')
res = res[out_blob]
print(res)

del exec_net
del plugin

[ INFO ] loading network files: 
	ir_model/ir_model.xml
	ir_model/ir_model.bin
[ INFO ] Preparing input blobs
1 1 28 28
[ INFO ] Loading model to the plugin
[ INFO ] Starting inference (1 iterations)
[ INFO ] Average running time of one iteration: 5.878448486328125 ms
[ INFO ] Processing output blob
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
